In [ ]:
import json
import requests
from bs4 import BeautifulSoup

In [ ]:
# Define the path to your .bib file
file_path = 'resonances.bib'

# Initialize an empty list to store the BibTeX keys
bibtex_keys = []

# Open and read the .bib file
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            # Check if the line starts with '@'
            if line.startswith('@'):
                # Extract the BibTeX key by splitting the line and removing unwanted characters
                key = line.split('{')[1].split(',')[0].strip()
                bibtex_keys.append(key)
except FileNotFoundError:
    print(f"The file {file_path} does not exist.")
except Exception as e:
    print(f"An error occurred: {str(e)}")

In [ ]:
# Filter the keys that start with 'resonances:'
resonance_keys = [key for key in bibtex_keys if key.startswith("resonances:")]

# Display the keys to be excluded from checking with INSPIRE
print("Keys to be excluded from INSPIRE check:")
print(resonance_keys)

# Get the keys that need to be checked with INSPIRE
to_check_with_inspire = [key for key in bibtex_keys if not key.startswith("resonances:")]

In [ ]:
def check_inspire(bibtex_key):
    # Make a request to the API
    response = requests.get(f"https://inspirehep.net/api/literature?q={bibtex_key}")
    data = json.loads(response.text)  # Parse the JSON response
    
    # Check if the result is unique
    if len(data["hits"]["hits"]) == 1:
        # Extract the record ID from the response
        record_id = data["hits"]["hits"][0]["metadata"]["control_number"]
        
        # Retrieve the BibTeX note
        bibtex_url = f"https://inspirehep.net/api/literature/{record_id}?format=bibtex"
        bibtex_response = requests.get(bibtex_url)
        bibtex_data = BeautifulSoup(bibtex_response.text, 'html.parser').get_text(strip=True)
        
        # Check if the key has changed
        new_key = bibtex_data.split('{', 1)[1].split(',', 1)[0].strip()
        if new_key != bibtex_key:
            print(f"{bibtex_key}: Key has changed to {new_key} in the new INSPIRE data.")
        
        # Display the BibTeX note
        # print(f"{bibtex_key}: found new bibtex item")
        return bibtex_data  # Return the BibTeX data
    else:
        print(f"{bibtex_key}: the result is not unique or no result found, len={len(data['hits']['hits'])}")
        return None  # Return None if the result is not unique or not found

In [ ]:
# Example usage
# bibtex_keys_to_check = ["Gounaris:1968mw", "JPAC:2021rxu", "Mai:2022eur"]
collected_bibtex_data = []

# Check each key with INSPIRE and collect the BibTeX data
for key in to_check_with_inspire:
    bibtex_item = check_inspire(key)
    if bibtex_item is not None:
        collected_bibtex_data.append(bibtex_item)

In [ ]:
# Display all collected BibTeX data
# print("\nAll collected BibTeX data:")
# print("\n\n".join(collected_bibtex_data))

In [ ]:
# Your existing code...

# Write the collected BibTeX data to a file
output_file_path = 'ref_update_resonances.bib'
try:
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write("\n\n".join(collected_bibtex_data))
    print(f"\nThe collected BibTeX data has been written to {output_file_path}")
except Exception as e:
    print(f"\nAn error occurred while writing to the file: {str(e)}")